# Gaussian elimination and backsubstitution

Solucionamos sistemas de la forma:

$$\begin{bmatrix}
        A_{00} & A_{01} & A_{02} & A_{03} \\
        A_{10} & A_{11} & A_{12} & A_{13} \\
        A_{20} & A_{21} & A_{22} & A_{23} \\
        A_{30} & A_{31} & A_{32} & A_{33} \\
    \end{bmatrix}
    \begin{bmatrix}
        w \\ x \\ y \\ z
    \end{bmatrix} =
    \begin{bmatrix}
        V_0 \\ V_1 \\ V_2 \\ V_3
    \end{bmatrix},$$

simplificándolos con eliminación gaussiana

$$\begin{bmatrix}
        1 & a_{01} & a_{02} & a_{03} \\
        0 & 1 & a_{12} & a_{13} \\
        0 & 0 & 1 & a_{23} \\
        0 & 0 & 0 & 1
    \end{bmatrix}
    \begin{bmatrix}
        w \\ x \\ y \\ z
    \end{bmatrix} =
    \begin{bmatrix}
        v_0 \\ v_1 \\ v_2 \\ v_3
    \end{bmatrix},$$
    
y luego resolviéndolos con backsubstitution

$$\begin{eqnarray}
z &=& v_3\\
y &=& v_2 - a_{23} z \\
x &=& v_1 - a_{12} y - a_{13} z \\
w &=& v_0 - a_{01} x - a_{02} y - a_{03} z 
\end{eqnarray}$$

In [1]:
import numpy as np

In [2]:
def Gauss_Elim_Backsub(A, v, ϵ = 0):
    # Tamaño del array v
    N = len(v)
    x = np.empty(N, float)
    
    for n in range(N):
        div = A[n,n]
        
        i = 1
        while abs(div) <= ϵ and i < N - n:
            A[n+i,:], A[n,:] = np.copy(A[n,:]), np.copy(A[n+i,:])
            v[n+i], v[n] = v[n], v[n+i]
            div = A[n,n]
            i += 1
        
        if div == 0 and i == N - n: 
            print("Hay algo mal aquí")
            return np.zeros_like(v)
        
        # División por cada elemento de la diagonal
        A[n,:] /= div
        v[n] /= div
        
        # Resta por cada fila
        for i in range(n+1,N):
            mult = A[i,n]
            A[i,:] -= mult*A[n,:]
            v[i] -= mult*v[n]
    
    # Backsubstituting
    for n in range(N-1,-1,-1):
        x[n] = v[n]
        for i in range(n+1,N):
            x[n] -= A[n,i]*x[i]

    return x

In [23]:
# Matriz A y array v

A = np.array([[2, 1, 4, 1],
             [3, 4, -1, -1],
             [1, -4, 1, 5],
             [2, -2, 1, 3]], float)
v = np.array([-4, 3, 9, 7], float)

In [18]:
# Usamos la función para calcular la solución

x = Gauss_Elim_Backsub(np.copy(A), np.copy(v))
x

array([ 1.61904762, -0.42857143, -1.23809524,  1.38095238])

In [19]:
# Solución con numpy

np.linalg.solve(A,v)

array([ 1.61904762, -0.42857143, -1.23809524,  1.38095238])

In [12]:
A

array([[ 1. ,  0.5,  2. ,  0.5],
       [ 0. ,  1. , -2.8, -1. ],
       [-0. , -0. ,  1. , -0. ],
       [-0. , -0. , -0. ,  1. ]])

In [13]:
v

array([-2. ,  3.6, -2. ,  1. ])

# Descomposición LU

In [20]:
def LU_descomposition(A):
    M, N = A.shape
    L = np.zeros_like(A,dtype=float) # Matriz L
    U = np.copy(A) # Matriz U
    
    ''' Realizamos la eliminación gaussiana de A
        Y guardamos esto como U 
        Para L, escogemos los elementos adecuados
        tras cada paso ''' 
    
    for n in range(N):
        for i in range(n,N):
            L[i,n] = U[i,n]
        
        # Eliminación gaussiana
        div = U[n,n]
        U[n,:] /= div
        
        for i in range(n+1,N):
            mult = U[i,n]
            U[i,:] -= mult*U[n,:]
    
    return L, U

In [21]:
def sol_LU(A, v):
    L, U = LU_descomposition(A)
    N = len(v)
    y = np.empty(N,dtype=float)
    x = np.empty(N,dtype=float)
    
    for n in range(0,N):
        y[n] = v[n]/L[n,n]
        for i in range(0,n):
            y[n] -= y[i]*L[n,i]/L[n,n]
    
    for n in range(N-1,-1,-1):
        x[n] = y[n]/U[n,n]
        for i in range(n+1,N):
            x[n] -= x[i]*U[n,i]/U[n,n]
            
    return x

In [24]:
L, U = LU_descomposition(A)
np.dot(L,U)

array([[ 2.,  1.,  4.,  1.],
       [ 3.,  4., -1., -1.],
       [ 1., -4.,  1.,  5.],
       [ 2., -2.,  1.,  3.]])

In [25]:
x = sol_LU(A, v)
x

array([ 2., -1., -2.,  1.])

In [26]:
# Del paquete numpy

x = np.linalg.solve(A,v)
x

array([ 2., -1., -2.,  1.])

# Inversa de una matriz

$$\mathbf{A x} = \mathbf{v}$$

$$\mathbf{x} = \mathbf{A^{-1} v}$$

In [28]:
# Del paquete numpy, la inversa de una matriz A
# se calcula con np.linalg.inv(A)

x = np.dot(np.linalg.inv(A),v)
x

array([ 2., -1., -2.,  1.])